In [8]:
import os
import json

from xcube.core.store import new_data_store
from pystac_client import Client
import pystac
import requests 
from collections import defaultdict

In [9]:
s3_store = new_data_store(
    "s3",
    root=os.environ["S3_USER_STORAGE_BUCKET"],
    max_depth=10,
    storage_options=dict(
        anon=False,
        key=os.environ["S3_USER_STORAGE_KEY"],
        secret=os.environ["S3_USER_STORAGE_SECRET"],
    ),
)

In [10]:
ds = s3_store.open_data("cubes/science/0.0.2/000000.zarr")
ds

<xarray.Dataset> Size: 58GB
Dimensions:                   (angle: 2, angle_x: 2, angle_y: 2, band: 12,
                               time: 1153, y: 1000, x: 1000, time_era5: 2983,
                               time_esa_wc: 2, time_lccs: 7)
Coordinates:
  * angle                     (angle) <U7 56B 'Zenith' 'Azimuth'
  * angle_x                   (angle_x) float64 16B 5.967e+05 6.017e+05
  * angle_y                   (angle_y) float64 16B 5.66e+06 5.655e+06
  * band                      (band) <U5 240B 'B01' 'B02' 'B03' ... 'B11' 'B12'
  * time                      (time) datetime64[ns] 9kB 2016-11-01T10:32:02.0...
  * time_era5                 (time_era5) datetime64[ns] 24kB 2016-11-01 ... ...
  * time_esa_wc               (time_esa_wc) datetime64[ns] 16B 2020-01-01 202...
  * time_lccs                 (time_lccs) datetime64[ns] 56B 2016-01-01 ... 2...
  * x                         (x) float64 8kB 5.967e+05 5.967e+05 ... 6.067e+05
  * y                         (y) float64 8kB 5.665e+06 5.665e+06 ... 5.655e+06
Data variables: (12/113)
    cloud_mask                (time, y, x) uint8 1GB dask.array<chunksize=(20, 500, 500), meta=np.ndarray>
    dem                       (y, x) float32 4MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    era5_d2m_max              (time_era5) float32 12kB dask.array<chunksize=(2983,), meta=np.ndarray>
    era5_d2m_mean             (time_era5) float32 12kB dask.array<chunksize=(2983,), meta=np.ndarray>
    era5_d2m_median           (time_era5) float32 12kB dask.array<chunksize=(2983,), meta=np.ndarray>
    era5_d2m_min              (time_era5) float32 12kB dask.array<chunksize=(2983,), meta=np.ndarray>
    ...                        ...
    lccs_processed_flag       (time_lccs, y, x) float32 28MB dask.array<chunksize=(7, 500, 500), meta=np.ndarray>
    s2l2a                     (band, time, y, x) float32 55GB dask.array<chunksize=(12, 20, 500, 500), meta=np.ndarray>
    scl                       (time, y, x) uint8 1GB dask.array<chunksize=(20, 500, 500), meta=np.ndarray>
    solar_angle               (time, angle, angle_y, angle_x) float32 37kB dask.array<chunksize=(1, 2, 2, 2), meta=np.ndarray>
    spatial_ref               int64 8B ...
    viewing_angle             (time, band, angle, angle_y, angle_x) float32 443kB dask.array<chunksize=(1, 12, 2, 2, 2), meta=np.ndarray>
Attributes: (12/36)
    acknowledgment:               DeepFeatures project
    affine_transform:             [10.0, 0.0, 596716.0, 0.0, -10.0, 5664658.0...
    bbox_utm:                     [596721, 5654663, 606711, 5664653]
    bbox_wgs84:                   [10.379465602333127, 51.035364882577994, 10...
    center_utm:                   [5659658.217015095, 601716.1995742661]
    center_wgs84:                 [51.079407, 10.452089]
    ...                           ...
    site_id:                      0
    time_range_end:               2024-12-31
    time_range_start:             2016-11-01
    utm_zone:                     32U
    version:                      0.0.2
    xcube_stac_attrs:             {'data_url': 'https://documentation.dataspa...

In [11]:
catalog = Client.open('https://stac.dataspace.copernicus.eu/v1')

## Get cloud cover for each time step

In [12]:
mapping = ds.s2l2a.attrs["xcube_stac_attrs"]["stac_item_ids"]

In [13]:
def access_item(data_id: str) -> pystac.Item:
    """Access item for a given data ID.

    Args:
        data_id: An identifier of data that is provided by this store.

    Returns:
        item object

    Raises:
        DataStoreError: Error, if the item json cannot be accessed.
    """
    url = f"https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/{data_id}"
    response = requests.request(
        method="GET",
        url=url
    )
    if response.ok:
        return pystac.Item.from_dict(
            json.loads(response.text),
            href=url,
            root=catalog,
            preserve_dict=False,
        )
    else:
        raise DataStoreError(response.raise_for_status())

In [16]:
cloud_covers = defaultdict(list)

for idx, (dt, item_ids) in enumerate(mapping.items()):
    if idx < 100:
        for item_id in item_ids:
            item = access_item(item_id)
            cloud_covers[dt].append(item.properties["eo:cloud_cover"])

In [20]:
for dt, cloud_cover in cloud_covers.items():
    print(dt, cloud_cover)

2016-11-01T10:32:02.026000 [34.74, 40.61]
2016-11-08T10:22:32.026000 [99.97, 98.65]
2016-11-11T10:32:52.026000 [99.71, 99.84]
2016-11-18T10:23:22.026000 [96.8, 85.52]
2016-11-21T10:33:32.026000 [98.47, 100.0]
2016-11-28T10:23:52.026000 [14.78, 22.0]
2016-12-01T10:34:12.026000 [100.0, 100.0]
2016-12-08T10:24:22.026000 [47.56, 87.79]
2016-12-11T10:34:32.026000 [93.37, 87.02]
2016-12-18T10:24:32.026000 [90.67, 80.99]
2016-12-21T10:34:32.026000 [98.29, 69.67]
2016-12-28T10:24:32.026000 [99.99, 94.76]
2016-12-31T10:34:32.026000 [23.96, 4.42]
2017-01-07T10:24:01.026000 [99.86, 99.71]
2017-01-10T10:34:11.026000 [98.37, 64.53]
2017-01-27T10:23:01.026000 [40.73, 0.05]
2017-01-30T10:32:51.026000 [99.78, 99.77]
2017-02-06T10:22:11.026000 [99.91, 100.0]
2017-02-09T10:31:51.026000 [99.96, 100.0]
2017-02-16T10:21:01.026000 [26.74, 21.64]
2017-02-19T10:30:51.026000 [92.6, 99.68]
2017-02-26T10:20:21.026000 [56.54, 85.43]
2017-03-01T10:30:21.026000 [84.14, 83.26]
2017-03-08T10:20:21.026000 [100.0, 100.

## Query according to cloud cover

In [19]:
items = search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=[9, 52, 11, 54],
    datetime="2024-01-01/2024-12-31",
    filter_lang="cql2-text",
    filter='eo:cloud_cover < 70'
).items()

list(items)

APIError: {"detail":[{"type":"dict_type","loc":["body","filter"],"msg":"Input should be a valid dictionary","input":"eo:cloud_cover < 70"},{"type":"literal_error","loc":["body","filter-lang"],"msg":"Input should be 'cql-json' or 'cql2-json'","input":"cql2-text","ctx":{"expected":"'cql-json' or 'cql2-json'"}}],"body":{"bbox":[9.0,52.0,11.0,54.0],"datetime":"2024-01-01T00:00:00Z/2024-12-31T23:59:59Z","collections":["sentinel-2-l2a"],"filter":"eo:cloud_cover < 70","filter-lang":"cql2-text"}}